# USLCI JSON-LD import

## 1. Import relevant packages

In [1]:
import bw2data as bd
import bw2calc as bc
import bw2io as bi
import bw_processing # Not sure yet if I need this
import bw_migrations # Not sure yet if I need this
#import bw_processing

import os
import numpy as np
import pandas as pd

## 2. Create/set the working folder:

In [2]:
bd.projects.dir

PosixPath('/Users/mmendez/Library/Application Support/Brightway3/default.c21f969b5f03d33d43e04f8f136e7682')

In [3]:
bd.projects.set_current('USLCI_FY22_Q2_2')

Exporter import

In [81]:
from bw2io.extractors.json_ld import JSONLDExtractor

In [82]:
path0 = '../7_BW2_Importing_USLCI/databases/FY22_Q2_01_Zolca_LCIA_methods_mapping_FEDEFL_3' # For some reason if I use the relative path it throws me an error
data = JSONLDExtractor.extract(path0)

## 3. Import JSON-LD:

In [4]:
path = '../7_BW2_Importing_USLCI/databases/FY22_Q2_01_Zolca_LCIA_methods_mapping_FEDEFL_3'
uslci = bi.importers.JSONLDImporter(
    path, 
    "USLCI_FY22_Q2_1", 
    preferred_allocation="PHYSICAL_ALLOCATION" # Most USLCI allocations are based on physical allocations (source: Rebe Feraldi)
)

### 3.1 Debugging the processes that have AvoidedProduct = True and input = True

OpenLCI sets avoided products (AvoidedProducts = True) as inputs (input = True) and they are always outputs. This issue gives an error while applying strategies `apply_strategies()`, specifically it gives the error using the function `json_ld_label_exchange_type`, this is fixed in cell [7].

In [5]:
uslci.data.keys()

#[e for e in uslci.data if e['name'] == 'Electricity, at eGrid, SRVC, 2010'][0]

dict_keys(['lcia_methods', 'currencies', 'lcia_categories', 'processes', 'dq_systems', 'locations', 'actors', 'flow_properties', 'sources', 'unit_groups', 'categories', 'flows'])

In [6]:
for i, l in enumerate(uslci.data['processes']['2a78de43-fdf2-4c5f-b527-89db6568ace8']['exchanges'][:2]):
    if l['avoidedProduct'] == True:
        print(i, l['flow']['name'])
        l['input'] = False
        uslci.data['processes']['2a78de43-fdf2-4c5f-b527-89db6568ace8']['exchanges'][i] = l

1 Natural gas, combusted in industrial boiler


In [7]:
for process_key, process_values in uslci.data['processes'].items():
    for i, exchange in enumerate(process_values['exchanges']):
        if (exchange['avoidedProduct'] == True) & (exchange['input'] == True):
            print(exchange['input'], exchange['flow']['name'])
            exchange['input'] = False
            uslci.data['processes'][process_key]['exchanges'][i] = exchange

True Natural gas, at processing, composite, kg
True Natural gas, combusted in industrial boiler
True Residual fuel oil, combusted in industrial boiler
True Natural gas, combusted in industrial boiler
True Residual fuel oil, combusted in industrial boiler
True Natural gas, combusted in industrial boiler
True Natural gas, combusted in industrial boiler
True Natural gas, combusted in industrial boiler
True Natural gas, combusted in industrial boiler
True Sold electricity, from industry-average containerboard
True Tall oil, from industry-average containerboard
True Turpentine, from industry-average containerboard
True Natural gas, combusted in industrial boiler


### 3.2. Checking the number of processes and flows in the USLCI

I use these cells to investigate and count flows and processes. I need to know how many processes, elemental, waste and production flows there are because even though I fixed the JSON-LD importer and managed to import USLCI and link the activities, it cannot run the LCA because the matrix is not square. There are 566 processes and 927 processes. I believe the extra processes are mislabeled flows.

In [8]:
len(uslci.data['processes']), len(uslci.data['flows'])


(454, 4168)

In [9]:
total_prod =0
for process_key, process_values in uslci.data['processes'].items():
    prod_n = 0
    for i, exc in enumerate(process_values['exchanges']):
        if (exc["flow"]["flowType"] == "PRODUCT_FLOW") & (~exc["input"]) & (exc.get('quantitativeReference') is True):
            prod_n += 1
    if prod_n > 1:
        print('{} has {} products'.format(process_values['name'], prod_n))
    total_prod += prod_n

    #print('{} has {} products'.format(process_values['name'], prod_n))
print('There are {} products'.format(total_prod))

There are 454 products


So, according to this, we have 454 products and productions. So the matrix should be square! Brightway is adding more production flows than it should. Let us fix that! I am suspecting the dummy flows are the issue. Also, it seems like waste flows are labeled as production.

Now let's check how many processes has waste as output:

In [10]:
process_n = 0
for process_key, process_values in uslci.data['processes'].items():
    prod_n = 0
    for i, exc in enumerate(process_values['exchanges']):
        if (exc["flow"]["flowType"] == "WASTE_FLOW") & (~exc["input"]):
            prod_n += 1
    if prod_n > 0:
        process_n += 1
        print('{} has {} output waste flows.'.format(process_values['name'], prod_n))
print('There are {} processes that have waste as outputs.'.format(process_n))

Natural gas, at extraction, coal bed methane, kg has 1 output waste flows.
Natural soda ash (Sodium carbonate), at plant has 4 output waste flows.
Polyethylene, high density, HDPE, virgin resin, at plant has 7 output waste flows.
Asphalt binder, 0.5% polyphosphoric acid (PPA), consumption mix, at terminal, from crude oil, 0.5% polyphosphoric acid has 10 output waste flows.
Ethylene glycol, materials production, organic compound, at plant, kg has 4 output waste flows.
Aluminum, cold rolling, at plant has 11 output waste flows.
Polypropylene, PP, virgin resin, at plant has 5 output waste flows.
Sodium hydroxide; chlor-alkali average, membrane cell; at plant; 50% solution state has 1 output waste flows.
Steel, stainless 304, scrap has 10 output waste flows.
Aluminum, hot rolling, at plant has 10 output waste flows.
Propylene, materials production, organic compound, at plant, kg has 6 output waste flows.
Steel, stainless 304, flat rolled coil has 10 output waste flows.
Ethylene, materials 

Not necessary but let's check how many processes has waste as input:

In [11]:
process_n = 0
for process_key, process_values in uslci.data['processes'].items():
    prod_n = 0
    for i, exc in enumerate(process_values['exchanges']):
        if (exc["flow"]["flowType"] == "WASTE_FLOW") & (exc["input"]):
            prod_n += 1
    if prod_n > 0:
        process_n += 1
        print('{} has {} input waste flows.'.format(process_values['name'], prod_n))
print('There are {} processes that have waste as inputs.'.format(process_n))

Asphalt binder, 8% ground rubber tire (GRT), consumption mix, at terminal, from crude oil, 8% ground rubber tire has 1 input waste flows.
Coil, coating, m2, at plant has 1 input waste flows.
Metal panel, insulated, at plant has 1 input waste flows.
There are 3 processes that have waste as inputs.


Is any of these flows labeled as quantitativeReference?

In [12]:
total_waste =0
for process_key, process_values in uslci.data['processes'].items():
    prod_n = 0
    for i, exc in enumerate(process_values['exchanges']):
        if (exc["flow"]["flowType"] == "WASTE_FLOW") & (~exc["input"]) & (exc.get('quantitativeReference') is True):
            prod_n += 1
    if prod_n > 0:
        print('{} has {} waste flows'.format(process_values['name'], prod_n))
    total_waste += prod_n

    #print('{} has {} products'.format(process_values['name'], prod_n))
print('There are {} waste flows as quantitativeReference'.format(total_waste))

There are 0 waste flows as quantitativeReference


No, awesome! Let's check how many flows are biosphere:

In [13]:
elem_n = 0
for key, values in uslci.data['flows'].items():
    if values['flowType'] == 'ELEMENTARY_FLOW':
        elem_n += 1
print('There are {} elementary flows in total'.format(elem_n))

There are 2879 elementary flows in total


I think I need to look at the Brightway imported file and check how many processes have more than 1 production flows and which are these processes. Keep running these cells and I will see you back in section ADD SECTION

## 4. Apply strategies to map JSON-LD to Brightway2 schema:

These strategies adapts the JSON-LD schema with the Brightway2 schema.

In [14]:
uslci.apply_strategies()

Applying strategy: json_ld_allocate_datasets
Applying strategy: json_ld_get_normalized_exchange_locations
Applying strategy: json_ld_convert_unit_to_reference_unit
Applying strategy: json_ld_get_activities_list_from_rawdata
Applying strategy: json_ld_add_products_as_activities
Applying strategy: json_ld_get_normalized_exchange_units
Applying strategy: json_ld_add_activity_unit
Applying strategy: json_ld_rename_metadata_fields
Applying strategy: json_ld_location_name
Applying strategy: json_ld_remove_fields
Applying strategy: json_ld_fix_process_type
Applying strategy: json_ld_label_exchange_type
Applying strategy: json_ld_prepare_exchange_fields_for_linking
Applying strategy: add_database_name
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: normalize_units
Applied 17 strategies in 5.94 seconds

	Created 2879 biosphere flows in separate database 'USLCI_FY22_Q2_1 biosphere'.
	Use either `.merge_biosphere_flows()` or `.write_separat

### 4.1. I did some debugging in the background...

What a beautiful feeling, all strategies were applied with no issues. I had to fix certain bugs mostly:

- `Applying strategy: json_ld_location_name`, error `KeyError: ['location']`, not all processes have locations, since Brightway2 requires locations it throws an error because it does not find the needed key. I identified these processes and set the location 'Northern America'. This might not be the right solution since it requires changing the database. I will contact Rebe Feraldi and inquiry about some of the processes not having location. It might be wiser to change the BW2 code so it does not require location.
- `Applying strategy: json_ld_location_name`, There was also a weird bug, where some of the locations were set as `None` and the 
- `Applying strategy: json_ld_add_activity_unit`, throws an error `AssertionError: Failed allocation`. This function identifies `PRODUCT_FLOW` in the outputs so it can get the units of the reference flow, however, there are processes with more than one `PRODUCT_FLOW`. I think what it is forcing is to have one as the "reference flow" thus it might be confusing reference flow with production flow. I changed the code increasing the `if` statement adding `quantitativeReference`, so it knows that which flow is the reference product to 'borrow' its units. The fix: added `& (exc.get('quantitativeReference') is True`.
- `Applying strategy: json_ld_label_exchange_type`, gives me `ValueError: Avoided products are outputs, not inputs.` while applying strategy json_ld_label_exchange_type (solved in section 3.1).This error comes up again, because
- `Applying strategy: json_ld_label_exchange_type`, giver the error `ValueError: Inputs must be products`, this is not necessarily true, because there are some `WASTE_FLOWS` that are inputs, so I add `WASTE_FLOWS` to be technosphere inputs!

### 4.2. Create dummy processes to link processes

Now let's check some statistics, what we mostly care about is that everything is linked!

In [15]:
uslci.statistics()

1802 datasets
52409 exchanges
488 unlinked exchanges
  Type substitution: 6 unique unlinked exchanges
  Type technosphere: 2 unique unlinked exchanges
  Type waste: 29 unique unlinked exchanges


(1802, 52409, 488)

This is expected, there are certain exchanges unlinked because the are production flows that go nowhere. Only emissions and resources are entitled to do this. So, what do we do? We assign dummy processes. There is a function in the brightway-io package, but I had to tweak it to make it work (`if "input" not in exc or "amount" not in exc:` added because it was throwing me a weird error). After fixing it, let's apply the dummy-maker strategy:

In [16]:
uslci.apply_strategy(bi.strategies.special.add_dummy_processes_and_rename_exchanges)

Applying strategy: add_dummy_processes_and_rename_exchanges


Let's apply the statistics again and...uslci.statistics()

In [17]:
uslci.statistics()

1838 datasets
52445 exchanges
0 unlinked exchanges
  


(1838, 52445, 0)

Boom! No unlinked exchanges.

## 5. Writing and saving the database

### 5.1. Check database dictionaries:

Now that we have uploaded the USLCI database, let's write the databases dictionary. If you paid attention, after applying the strategies, there is a message at the end of the output that reads the following:

```
Created 2879 biosphere flows in separate database 'USLCI_FY22_Q2_1 biosphere'.
Use either `.merge_biosphere_flows()` or `.write_separate_biosphere_database()` to write these flows.
```

This means that it has splitted the USLCI into a different database. This is because it will use this database to generate the biosphere matrix. It also makes easier to query processes and flows.
You can check which databases are within the project using `bd.databases`. If this is the first time you run this code, there should be 0 objects in the database dictionary, if there are already written databases.

In [18]:
bd.databases # If this is the first time you run this code, there should be 0 objects in the database dictionary:

Databases dictionary with 2 object(s):
	USLCI_FY22_Q2_1
	USLCI_FY22_Q2_1 biosphere

Let's fill out our project folder with some databases!

#### 5.1.1. Write the biosphere database:

In [19]:
uslci.write_separate_biosphere_database()

Vacuuming database 


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 09/20/2022 15:55:34
  Finished: 09/20/2022 15:55:34
  Total time elapsed: 00:00:00
  CPU %: 2.40
  Memory %: 1.58
Created database: USLCI_FY22_Q2_1 biosphere


#### 5.1.2.Write the technosphere database:

In [20]:
uslci.write_database()

Not able to determine geocollections for all datasets. This database is not ready for regionalization.


Writing activities to SQLite3 database:


Vacuuming database 


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


Title: Writing activities to SQLite3 database:
  Started: 09/20/2022 15:55:43
  Finished: 09/20/2022 15:55:46
  Total time elapsed: 00:00:02
  CPU %: 2.30
  Memory %: 1.59
Created database: USLCI_FY22_Q2_1


Brightway2 SQLiteBackend: USLCI_FY22_Q2_1

<div class="alert alert-block alert-warning">
<b>To fix:</b>
<code>
Not able to determine geocollections for all datasets. This database is not ready for regionalization.
</code>
Indicates that the locations given are not within the brightway2 geocollections. This is a problem for future Mac, but it needs to be fixed, probably by mapping the location names properly with brightway location dictionary.
</div>

Now you should see the uploaded database dictionaries:

In [21]:
bd.databases

Databases dictionary with 2 object(s):
	USLCI_FY22_Q2_1
	USLCI_FY22_Q2_1 biosphere

### 5.2. Saving the databases

Hoorraaay! Now let's be tidy and save them in variables for easy access:

In [22]:
bio = bd.Database('USLCI_FY22_Q2_1 biosphere')

In [23]:
db = bd.Database('USLCI_FY22_Q2_1')

### 5.3. Querying the databases

#### 5.3.1 Let's look at the biosphere database:

You can search activities using list comprehension:

In [24]:
carbon_query = [bio_flow for bio_flow in bio if bio_flow['name'].lower().startswith('carbon')] # Use .lower() to make it non-case sensitive
carbon_query; # Remove ';' if you want to reveal the output!

In [25]:
print('There are {} carbon-related flows!'.format(len(carbon_query)))

There are 46 carbon-related flows!


You can also use `.search()` to find the flow:

In [26]:
bio.search('carbon dioxide')

['Carbon dioxide' (, None, ('Elementary Flows', 'emission', 'air', 'stratosphere')),
 'Carbon dioxide' (, None, ('Elementary Flows', 'emission', 'air', 'troposphere', 'rural')),
 'Carbon dioxide' (, None, ('Elementary Flows', 'resource', 'air')),
 'Carbon dioxide' (, None, ('Elementary Flows', 'emission', 'air', 'troposphere', 'very high')),
 'Carbon dioxide' (, None, ('Elementary Flows', 'emission', 'air', 'troposphere', 'urban')),
 'Carbon dioxide' (, None, ('Elementary Flows', 'resource', 'air', 'troposphere')),
 'Carbon dioxide' (, None, ('Elementary Flows', 'emission', 'ground')),
 'Carbon dioxide' (, None, ('Elementary Flows', 'emission', 'air'))]

Let's refine our search and save one of these in a variable:

In [27]:
carboncete = [act for act in bio if 'Carbon dioxide' in act['name']
                                            and 'Elementary Flows' in str(act['categories'])
                                            and 'emission' in str(act['categories'])
                                            and 'ground' in str(act['categories'])][0]

#### 5.3.2. Now let's look at our technosphere database

The technosphere database is composed of activities thaat can be processes (they have exchanges, input and output flows) and product flows (these can be outputs or inputs). Let's explore a different way to select an activity... let's say I am just testing and I just want a random activity. Can I do it? Selvfølgelig! Actio `.random()`!!

In [28]:
random_act = db.random()
random_act

'Sodium chlorate, at plant' (, None, ('Technosphere Flows', 'CUTOFF Flows'))

Let's look at it with more intensity...

In [29]:
random_act.as_dict();

Looking at this closely, you can see that there are no exchanges, this is because these are saved in a different location. Let's now look at a specific process:

In [30]:
db.search('corn')

['Corn, production' (, Northern America, ('Technosphere Flows', '11: Agriculture, Forestry, Fishing and Hunting', '1111: Oilseed and Grain Farming')),
 'Corn, decomposition' (, Northern America, ('Technosphere Flows', '11: Agriculture, Forestry, Fishing and Hunting', '1111: Oilseed and Grain Farming')),
 'Corn, at field' (, Northern America, ('Technosphere Flows', '11: Agriculture, Forestry, Fishing and Hunting', '1111: Oilseed and Grain Farming')),
 'Pesticide, corn' (, Northern America, ('Technosphere Flows', '11: Agriculture, Forestry, Fishing and Hunting', '1151: Support Activities for Crop Production')),
 'Fertilizer, corn' (, Northern America, ('Technosphere Flows', '11: Agriculture, Forestry, Fishing and Hunting', '1151: Support Activities for Crop Production')),
 'Seed corn' (, None, ('Technosphere Flows', '11: Agriculture, Forestry, Fishing and Hunting', '1111: Oilseed and Grain Farming')),
 'Corn stover, carted' (, Northern America, ('Technosphere Flows', '11: Agriculture, Fo

In [31]:
len(db.search('corn'))

25

You can also filter your search:

In [32]:
db.search('corn', filter={'categories': 'Technosphere'}) # Here I show the flows

Excluding 18 filtered results


['Corn, production' (, Northern America, ('Technosphere Flows', '11: Agriculture, Forestry, Fishing and Hunting', '1111: Oilseed and Grain Farming')),
 'Corn, decomposition' (, Northern America, ('Technosphere Flows', '11: Agriculture, Forestry, Fishing and Hunting', '1111: Oilseed and Grain Farming')),
 'Corn, at field' (, Northern America, ('Technosphere Flows', '11: Agriculture, Forestry, Fishing and Hunting', '1111: Oilseed and Grain Farming')),
 'Pesticide, corn' (, Northern America, ('Technosphere Flows', '11: Agriculture, Forestry, Fishing and Hunting', '1151: Support Activities for Crop Production')),
 'Fertilizer, corn' (, Northern America, ('Technosphere Flows', '11: Agriculture, Forestry, Fishing and Hunting', '1151: Support Activities for Crop Production')),
 'Seed corn' (, None, ('Technosphere Flows', '11: Agriculture, Forestry, Fishing and Hunting', '1111: Oilseed and Grain Farming')),
 'Corn stover, carted' (, Northern America, ('Technosphere Flows', '11: Agriculture, Fo

You can also `mask`!

In [33]:
db.search('corn', mask={'categories': 'Technosphere'}) # Here I show the processes

Excluding 24 filtered results


['Corn steep liquor' (kilogram, Northern America, None),
 'Corn stover, carted' (kilogram, Northern America, None),
 'Tillage, intensive, corn production' (square meter, Northern America, None),
 'Corn, whole plant, at field' (kilogram, Northern America, None),
 'Corn, whole plant, at field' (kilogram, Northern America, None),
 'Corn, decomposition, 15.5% moisture' (kilogram, Northern America, None),
 'Corn stover, ground and stored' (kilogram, Northern America, None),
 'Corn, production, average, US, 2022' (kilogram, Northern America, None),
 'Corn grain, harvested and stored' (kilogram, Northern America, None),
 'Harvest, corn, single pass' (square meter, Northern America, None),
 'Lime, agricultural, corn production' (square meter, Northern America, None),
 'Tillage, conservation, corn production' (square meter, Northern America, None),
 'Ethanol, denatured, corn dry mill' (kilogram, Northern America, None),
 'Ethanol, denatured, corn dry mill' (kilogram, Northern America, None),
 '

It is also important to stress the importance of filtering, because the `search` function returns a maximum of 25 possible candidates. This is why I prefer list comprehension.

Weird, there are codes merged in one.

In [34]:
len([act['code'] for act in db if '.' in act['code']])

168

In [35]:
len([act['code'] for act in db if '.' not in act['code']])

1670

168 of them are behaving weird 1635 of them are normal. They did not drop many of the category names that are not necessary, on top of that, their code name is merges with a flow. Questions:
- What do these processes have in common?
- Why do they have double codes?
- Why aren't they dropping come of the items?

Let's start if all of them have `allocationFactors`

In [36]:
len([act['allocationFactors'] for act in db if '.' in act['code']])

168

All of these have allocationFactors. Sanity check!

In [37]:
len([act for act in db if 'allocationFactors' in act])

168

Ok, so these weirdos are the ones that have the key `allocationFactor`

Here I filter for the processes that have more than 15 entries and do not have a '.' in the code. I want to check if these are normal processes and why do they have so many entries:

In [38]:
len([act for act in db if len(act) > 15 and '.' not in act['code']])

0

In [39]:
[act.as_dict() for act in db if len(act) > 15 and '.' not in act['code']];

Ok, these processes did not drop some of the entries... so something does not work in the JSON-LD importer, these processes are escaping the mapping for some reason. However, their code is normal.

In [40]:
len([act.as_dict() for act in db if len(act) > 15])

0

198 processes have a looot of entries, the average entry number should be:

In [41]:
np.median([len(act.as_dict()) for act in db]), np.mean([len(act.as_dict()) for act in db]), np.std([len(act.as_dict()) for act in db])

(8.0, 8.686615886833515, 1.111198010191814)

10

Let's be more specific:

In [42]:
for entries in range(5, 19):
    activities = len([act.as_dict() for act in db if len(act) == entries])
    print('# of activities with {} entries: {}'.format(entries, activities))


# of activities with 5 entries: 0
# of activities with 6 entries: 0
# of activities with 7 entries: 36
# of activities with 8 entries: 1237
# of activities with 9 entries: 0
# of activities with 10 entries: 397
# of activities with 11 entries: 168
# of activities with 12 entries: 0
# of activities with 13 entries: 0
# of activities with 14 entries: 0
# of activities with 15 entries: 0
# of activities with 16 entries: 0
# of activities with 17 entries: 0
# of activities with 18 entries: 0


In [43]:
entry_list = list([7, 8, 10, 11])

In [44]:
for i in entry_list:
    activities = len([act.as_dict() for act in db if len(act) == i])
    keys = [act.as_dict().keys() for act in db if len(act) == i][0]
    processes = len([act.as_dict() for act in db if len(act) == i and act['type'] == 'process'])
    len_keys = len(keys)
    print('There are \033[1m{}\033[0m activities with \033[1m{}\033[0m entries. Its keys are: {}. There are a total of \033[1m{}\033[0m keys, of which \033[1m{}\033[0m are processes.\n'.format(activities, i, keys, len_keys, processes))

There are 36 activities with 7 entries. Its keys are: dict_keys(['name', 'database', 'code', 'classifications', 'location', 'type', 'id']). There are a total of 7 keys, of which 36 are processes.

There are 1237 activities with 8 entries. Its keys are: dict_keys(['code', 'name', 'categories', 'location', 'unit', 'type', 'database', 'id']). There are a total of 8 keys, of which 0 are processes.

There are 397 activities with 10 entries. Its keys are: dict_keys(['name', 'defaultAllocationMethod', 'location', 'filename', 'unit', 'code', 'classifications', 'type', 'database', 'id']). There are a total of 10 keys, of which 397 are processes.

There are 168 activities with 11 entries. Its keys are: dict_keys(['name', 'defaultAllocationMethod', 'location', 'allocationFactors', 'filename', 'code', 'unit', 'classifications', 'type', 'database', 'id']). There are a total of 11 keys, of which 168 are processes.



Let's check out the 7 entry process, it does not have a unit... why?

In [45]:
[act.as_dict() for act in db if len(act) == 7];

In [46]:
dummy = [act for act in db if len(act) == 7][0]
dummy

'medium radioactive wastes' (None, GLO, None)

In [47]:
[exc for exc in dummy.exchanges()]

[Exchange: 1 None 'medium radioactive wastes' (None, GLO, None) to 'medium radioactive wastes' (None, GLO, None)>]

Aaah it is the dummy process. Ok, it does not need units.

To drop some of these entries I just have to modify the functiondef json_ld_remove_fields(db) in the `brightway2-io/strategies/json_ld`

In [48]:
len([act for act in db if 'corn' in act['name'].lower()])

42

See? 42 entries now! Looking at this list, you can see that this search mixes processes and flows, let's find the process flows:

I will come back to this later...

### Let's manually add a LCIA method
This importer does not recognize the LCIA methods, therefore we need to add them manually. I will debug this later!

First, let's find the `Bad stuff` biosphere flow.

In [49]:
bad_stuff = [act for act in bio if act['name'] == 'Carbon dioxide'][0]


In [50]:
myLCIAdata = [[(bad_stuff['database'], bad_stuff['code']), 2.0]] # A method list needs: a reference to the flow: tuple (database, 'code')), a characterization factor number, and localization (if no localization is given, 'GLO' is used)
method_key = ('MacIM', 'Global warming', 'total')
my_method = bd.Method(method_key)
my_method.validate(myLCIAdata)
my_method.register()
my_method.write(myLCIAdata)

#### Now we define a functional unit:
This one might be a bit counterintuitive, our functional unit here is **Impact of assembling 5 bottles**, intuintively one would select the activity, but bw2 selects the flow coming out of the `Bottle assembly` activity (i.e. `Bottle`, which is a `product` not a `process`).

In [51]:
extr_alu = [act for act in db if act['name'].lower().startswith('c')][0]

In [52]:
functional_unit = {extr_alu : 5}

In [53]:
type(functional_unit)

dict

#### Run the LCA!

In [54]:
lca = bc.LCA(functional_unit, method_key) 

In [55]:
#bc.LeastSquaresLCA(lca)

In [56]:
lca.lci() 
lca.lcia()
lca.score
print(lca.inventory)

NonsquareTechnosphere: Technosphere matrix is not square: 601 activities (columns) and 962 products (rows). Use LeastSquaresLCA to solve this system, or fix the input data

Not square matrix. Let's find out which processes have more than one product:

In [57]:
n_activities = 0
n_processes = 0
n_flow = 0
m_bio = 0
    
for act in db:
    n_activities += 1
    if act['type'] == 'process':
        n_processes += 1
    elif act['type'] == 'product':
        n_flow += 1
    elif act['type'] == 'biosphere': # This is just a sanity check to see if there are some biosphere flows laying around.
        m_bio += 1
    
print('There are in total {} activities.'.format(n_activities))
print('Of which, {} are processess, {} are products (technosphere flows) and {} are biosphere flows.'.format(n_processes, n_flow, m_bio))

There are in total 1838 activities.
Of which, 601 are processess, 1237 are products (technosphere flows) and 0 are biosphere flows.


So... There are a total of 1803 activities. From those, 601 are processes and 1237 are products (i.e. technosphere flows)

Ok.... those are a bunch of products. My problem here is that I need only 601 production products to be linked with processes to make the matrix square. Let's see which processes have more processes as outputs. One of them is the waste flows, maybe I could create a dummy process that takes the waste flows so they are not hanging. I don't know...

In [58]:
production_counter = 0
for act in db:
    for exc in act.exchanges():
        if exc['type'] == 'production':
            production_counter += 1
            
print(production_counter)

565


I select a product that has a lot of waste outputs, let's see how these are labeled:

In [59]:
uslci.statistics()

1838 datasets
52445 exchanges
0 unlinked exchanges
  


(1838, 52445, 0)

In [60]:
natural_soda_ash = [act for act in db if act['code'] == '0d95cc8b-a9a0-3630-a760-1ab4d88257d8'][0]

In [61]:
len([exc for exc in natural_soda_ash.exchanges()])

80

Ok 80 exchanges, which ones are outputs?

In [62]:
[exc.as_dict().keys() for exc in natural_soda_ash.exchanges()][0]

dict_keys(['amount', 'unit', 'type', 'name', 'code', 'input', 'output'])

Interesting, there is a key called `output` which it should not have. Let's see what's inside:

In [63]:
[exc.as_dict() for exc in natural_soda_ash.exchanges()][0]

{'amount': 1000.0,
 'unit': 'kilogram',
 'type': 'production',
 'name': 'Natural soda ash (Sodium carbonate), at plant',
 'code': 'c0af8d39-2a03-3e1d-8548-67989e0fda5a',
 'input': ('USLCI_FY22_Q2_1', 'c0af8d39-2a03-3e1d-8548-67989e0fda5a'),
 'output': ('USLCI_FY22_Q2_1', '0d95cc8b-a9a0-3630-a760-1ab4d88257d8')}

- `'input': ('USLCI_FY22_Q2_1', 'c0af8d39-2a03-3e1d-8548-67989e0fda5a')`: Technosphere flow --> Natural soda ash (Sodium carbonate), at plant
- `'output': ('USLCI_FY22_Q2_1', '0d95cc8b-a9a0-3630-a760-1ab4d88257d8')`: Process --> Natural soda ash (Sodium carbonate), at plant

Interesting, the `output` is the process... I am not sure this makes sense because Brightway2 should only have `inputs`.

In [64]:
[exc for exc in natural_soda_ash.exchanges() if exc['type'] == 'production']

[Exchange: 1000.0  'Natural soda ash (Sodium carbonate), at plant' (, United States, ('Technosphere Flows', '21: Mining, Quarrying, and Oil and Gas Extraction', '2123: Nonmetallic Mineral Mining and Quarrying')) to 'Natural soda ash (Sodium carbonate), at plant' (kilogram, United States, None)>]

In [65]:
len([exc for exc in natural_soda_ash.exchanges() if exc['type'] == 'production'])

1

Aha, all these wastes are dummies... is there a way a can call them something different than `production`? Let's see if all the dummies I created are the ones making my life miserable :)

In [66]:
counter = 0
for exc in natural_soda_ash.exchanges():
    if exc['type'] == 'production':
        print(exc)
        counter += 1
print(counter)

Exchange: 1000.0  'Natural soda ash (Sodium carbonate), at plant' (, United States, ('Technosphere Flows', '21: Mining, Quarrying, and Oil and Gas Extraction', '2123: Nonmetallic Mineral Mining and Quarrying')) to 'Natural soda ash (Sodium carbonate), at plant' (kilogram, United States, None)>
1


In [67]:
uslci.statistics()

1838 datasets
52445 exchanges
0 unlinked exchanges
  


(1838, 52445, 0)

What are those flows that are not production?

In [68]:
for act in db:
    continue

In [69]:
act#['type']

'Forest residue, processed and loaded, at landing system' (, Northern America, ('Technosphere Flows', '11: Agriculture, Forestry, Fishing and Hunting', '1133: Logging'))

In [70]:
a = [[exc['type'] for exc in act.exchanges()] for act in db if act['type']=='process']
b = [item for sublist in a for item in sublist]

In [71]:
pd.Series(b).value_counts()

biosphere       44654
technosphere     6705
production        565
waste             507
substitution       14
dtype: int64

In [72]:
llave = [act.as_dict().keys() for act in db]

In [73]:
pd.Series(llave).value_counts()

(code, name, categories, location, unit, type, database, id)                                                               1237
(name, defaultAllocationMethod, location, filename, unit, code, classifications, type, database, id)                        397
(name, defaultAllocationMethod, location, allocationFactors, filename, code, unit, classifications, type, database, id)     168
(name, database, code, classifications, location, type, id)                                                                  36
dtype: int64

In [79]:
[act.get('unit') for act in db][:100]

['',
 '',
 'kilogram',
 'kilogram',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'kilogram',
 '',
 '',
 'kilogram',
 'kilogram',
 '',
 '',
 'megajoule',
 '',
 '',
 None,
 'ton kilometer',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 None,
 'kilogram',
 'unit',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'megajoule',
 '',
 '',
 '',
 'megajoule',
 '',
 'kilogram',
 'p*km',
 'megajoule',
 'p*km',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'p*km',
 '',
 '',
 '',
 'p*km',
 '',
 '',
 '',
 'ton kilometer',
 '',
 '',
 'ton kilometer',
 'kilogram',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'kilogram',
 'megajoule',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'kilogram',
 '',
 '',
 '',
 '']

In [74]:
[act.get('classifications') for act in db]

[None,
 {'@type': 'Category',
  '@id': '22b8f169-c337-3c11-98a7-cc688afb99b3',
  'name': '2211: Electric Power Generation, Transmission and Distribution',
  'categoryPath': ['22: Utilities'],
  'categoryType': 'Process'},
 None,
 None,
 None,
 None,
 {'@type': 'Category',
  '@id': '40500f84-eb6b-3e6d-80a0-90092b2ea35d',
  'name': '3341: Computer and Peripheral Equipment Manufacturing',
  'categoryPath': ['31-33: Manufacturing'],
  'categoryType': 'Process'},
 None,
 None,
 None,
 None,
 {'@type': 'Category',
  '@id': 'e032289e-6bca-36b2-876c-63e7c54d70d9',
  'name': '3261: Plastics Product Manufacturing',
  'categoryPath': ['31-33: Manufacturing'],
  'categoryType': 'Process'},
 {'@type': 'Category',
  '@id': 'ae58118a-9c50-3e6f-b450-1dfe17d94bc8',
  'name': '4841: General Freight Trucking',
  'categoryPath': ['48-49: Transportation and Warehousing'],
  'categoryType': 'Process'},
 {'@type': 'Category',
  '@id': 'ae58118a-9c50-3e6f-b450-1dfe17d94bc8',
  'name': '4841: General Freight T

In [75]:
[act.get('classifications') for act in db]

[None,
 {'@type': 'Category',
  '@id': '22b8f169-c337-3c11-98a7-cc688afb99b3',
  'name': '2211: Electric Power Generation, Transmission and Distribution',
  'categoryPath': ['22: Utilities'],
  'categoryType': 'Process'},
 None,
 None,
 {'@type': 'Category',
  '@id': 'cbcdb90f-432a-3a1b-a8ec-b2b8f369021d',
  'name': '3221: Pulp, Paper, and Paperboard Mills',
  'categoryPath': ['31-33: Manufacturing'],
  'categoryType': 'Process'},
 None,
 {'@type': 'Category',
  '@id': '9a1ae80e-2a4f-3fe1-8282-a284cc54c0c5',
  'name': '3313: Alumina and Aluminum Production and Processing',
  'categoryPath': ['31-33: Manufacturing'],
  'categoryType': 'Process'},
 None,
 {'@type': 'Category',
  '@id': '4da1e9cd-0458-3a61-a8f9-b04fa15e8699',
  'name': '2213: Water, Sewage and Other Systems',
  'categoryPath': ['22: Utilities'],
  'categoryType': 'Process'},
 None,
 None,
 {'disposal, solid waste, process, to sanitary landfill': 'dummy'},
 None,
 None,
 None,
 None,
 None,
 {'@type': 'Category',
  '@id': 